# 실습: Zero-shot Classification

이번 실습에서는 open LLM을 가지고 zero-shot classification을 해봅니다. 먼저 필요한 library들을 설치합시다.

In [1]:
# !pip install datasets
# !pip install openai
# !pip install openai==0.28
!pip install -U bitsandbytes
!pip install peft
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 3.6 MB/s eta 0:00:00




2023 수능 json 데이터 가져오기


In [2]:
import requests

url = "https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/master/data/2023_11_KICE.json"
response = requests.get(url)
data = response.json()
data[1]

{'id': '2023_11_KICE_4-9',
 'paragraph': '(가)[A](중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로 분류, 정리하여 이용에 편리하도록 편찬한서적이다. 일반적으로 유서는 기존 서적에서 필요한 부분을뽑아 배열할 뿐 상호 비교하거나 편찬자의 해석을 가하지 않았다. 유서는 모든 주제를 망라한 일반 유서와 특정 주제를다룬 전문 유서로 나눌 수 있으며, 편찬 방식은 책에 따라다른 경우가 많았다. 중국에서는 대체로 왕조 초기에 많은 학자를 동원하여 국가 주도로 대규모 유서를 편찬하여 간행하였다. 이를 통해 이전까지의 지식을 집성하고 왕조의 위엄을 과시할 수 있었다.고려 때 중국 유서를 수용한 이후, 조선에서는 중국 유서를활용하는 한편, 중국 유서의 편찬 방식에 ⓐ(따라) 필요에 맞게 유서를 편찬하였다. 조선의 유서는 대체로 국가보다 개인이 소규모로 편찬하는 경우가 많았고, 목적에 따른 특정주제의 전문 유서가 집중적으로 편찬되었다. 전문 유서 가운데 편찬자가 미상인 유서가 많은데, 대체로 간행을 염두에 두지 않고 기존 서적에서 필요한 부분을 발췌, 기록하여 시문 창작, 과거 시험 등 개인적 목적으로 유서를 활용하고자 하였기 때문이었다.)이 같은 유서 편찬 경향이 지속되는 가운데 17세기부터 실학의학풍이 하나의 조류를 형성하면서 유서 편찬에 변화가 나타났다. ㉮(실학자들의 유서)는 현실 개혁의 뜻을 담았고, 편찬 의도를 지식의 제공과 확산에 두었다. 또한 단순 정리를 넘어 지식을 재분류하여 범주화하고 평가를 더하는 등 저술의 성격을 드러냈다. 독서와 견문을 통해 주자학에서 중시되지 않았던 지식을 집적했고, 증거를 세워 이론적으로 밝히는 고증과 이에 대한 의견 등 ‘안설’을 덧붙이는 경우가 많았다. 주자학의 지식을 ⓑ(이어받는) 한편, 주자학이 아닌 새로운 지식을 수용하는 유연성과 개방성을 보였다. 광범위하게 정리한 지식을 식자층이 ⓒ(쉽게) 접할 수 있어야 한다고 생각했고, 객관적 사실 탐구를 중시하여 박물학과 자연 

그 다음 GPT4를 사용하기 위해 다음과 같은 작업을 진행합니다:
1. huggingface.co 계정 만들고 로그인하기
2. https://www.kaggle.com/models/google/gemma/license/consent 에서 Gemma license 동의하기
3. 홈 화면으로 돌아와, `Profile > Settings > Access Tokens` 메뉴로 들어와 "Write" type의 token 생성하기
4. 생성한 토큰을 아래 "HF TOKEN"에 불여넣고 셀을 실행하기.

In [3]:
from huggingface_hub import login
# from openai import OpenAI

from google.colab import userdata
import os
os.environ['HF_TOKEN'] = ''
# os.environ['OPENAI_API_KEY'] = ''

# login(userdata.get('HF_TOKEN'))
login(os.environ['HF_TOKEN'])
# OpenAI API 키 설정
# client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import warnings


# 8비트 양자화 설정
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)


model_name = "beomi/KoAlpaca-Polyglot-5.8B"


# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16,
    use_cache=False,  # 그래디언트 체크포인팅 활성화

)

# LoRA 설정
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA 적용
model = get_peft_model(model, config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00011-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

model-00012-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/515M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [43]:


def prediction(paragraph, problems):

    predicted_answers = []

    # 지문을 포함한 초기 프롬프트 생성
    initial_prompt = f"""다음은 2023년도 대학수학능력시험 국어 영역의 지문입니다. 이 지문을 기반으로 앞으로 여러 개의 질문에 답하게 될 것입니다. 지문을 잘 읽고 이해하세요.

    지문:
    {paragraph}
    """

    inputs = tokenizer(initial_prompt, return_tensors="pt", return_token_type_ids=False).to(model.device)

    # 지문 이해 확인
    with torch.no_grad(), warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        outputs = model.generate(**inputs, max_new_tokens=20, num_return_sequences=1, temperature=0.7, pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(paragraph)
    # 각 문제에 대한 답변 생성
    for problem in problems:
        question_prompt = f"""앞서 읽은 지문을 기반으로 다음 질문에 답하세요.

        질문: {problem['question']}

        선택지:
        """
        choices = ""
        for i, choice in enumerate(problem['choices'], 1):
            choices += f"{i}. {choice}\n"
        question_prompt += choices
        # question_prompt += "\n위 문제에대해 정답으로 가장 적절한 선택지의 번호만 쓰시오. 설명은 필요 없습니다."
        question_prompt += "\n질문과 선택지를 보고 정답으로 가장 적절한 선택지의 번호만을 작성하세요. 1에서 5 사이의 숫자만 답변으로 허용됩니다. 다른 설명은 필요 없습니다.\n\n답변: "

        inputs = tokenizer(question_prompt, return_tensors="pt", return_token_type_ids=False).to(model.device)

        with torch.no_grad(), warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning)
            # outputs = model.generate(**inputs, max_new_tokens=10, num_return_sequences=1, temperature=0.5, pad_token_id=tokenizer.eos_token_id)
            outputs = model.generate(
                **inputs,
                max_new_tokens=5,  # 출력 토큰 수를 제한
                num_return_sequences=1,
                temperature=0.5,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=2  # 같은 n-gram의 반복을 방지
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        predicted_answer = next((int(char) for char in response if char.isdigit() and 1 <= int(char) <= 5), None)
        print(f"response:{response}\n\n")
        print(f"111problem['question']:{problem['question']}")
        print(choices)
        print(f"predicted_answer:{predicted_answer}")
        predicted_answers.append(predicted_answer)

    torch.cuda.empty_cache()
    return predicted_answers


# def ensemble_prediction(paragraph, problems, n=3):
#     predictions = []
#     for _ in range(n):
#         pred = prediction(paragraph, problems)
#         predictions.append(pred)

#     # 각 문제에 대해 가장 많이 나온 답을 선택
#     final_predictions = []
#     for i in range(len(problems)):
#         answers = [p[i] for p in predictions]
#         final_predictions.append(max(set(answers), key=answers.count))

#     return final_predictions




In [56]:
!pip install openai==0.28

In [ ]:
import re
import time
import random
import openai
openai.api_key = ""


def api_call_with_retry(func, max_retries=5, *args, **kwargs):
    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except openai.error.RateLimitError as e:
            if attempt == max_retries - 1:
                raise
            wait_time = float(str(e).split('Please try again in ')[1].split('s.')[0])
            wait_time += random.uniform(0.1, 1.0)  # 추가 랜덤 대기 시간
            print(f"Rate limit reached. Waiting for {wait_time:.2f} seconds.")
            time.sleep(wait_time)

def prediction2(paragraph, problems):
    predicted_answers = []

    for problem in problems:
        time.sleep(0.5)
        prompt = f"""다음은 2023년도 대학수학능력시험 국어 영역의 지문과 질문입니다. 지문을 잘 읽고 질문에 답하세요.

        지문:
        {paragraph}

        질문: {problem['question']}

        선택지:
        """
        for i, choice in enumerate(problem['choices'], 1):
            prompt += f"{i}. {choice}\n"
        prompt += "\n위 문제의 정답으로 가장 적절한 것의 번호만을 숫자로 작성하세요. 반드시 1에서 5 사이의 숫자 하나만 답변으로 제시하세요. 어떠한 추가 설명도 하지 마세요."

        response = api_call_with_retry(
            openai.ChatCompletion.create,
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant specialized in Korean language exams. Always respond with a single number between 1 and 5, without any additional text."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1
        )

        answer = response.choices[0].message['content'].strip()
        print(f"Raw GPT-4 response: {answer}")

        # 응답에서 숫자만 추출
        match = re.search(r'\d', answer)
        if match:
            predicted_answer = int(match.group())
            if 1 <= predicted_answer <= 5:
                predicted_answers.append(predicted_answer)
            else:
                predicted_answers.append(None)
        else:
            predicted_answers.append(None)

    return predicted_answers


In [54]:
from tqdm.notebook import tqdm

# 모델 평가
model.eval()

total_score = 0
correct_answers = 0
total_questions = 0

total_problems = sum(len(question_set['problems']) for question_set in data)
total_answers = []

with tqdm(total=total_problems, desc="처리 중인 문제") as pbar:
    for num, question_set in enumerate(data):
        # if num == 1: break
        paragraph = question_set['paragraph']
        # predicted_answers = prediction(paragraph, question_set['problems'])  # 지문과 문제 리스트를 전달
        predicted_answers = prediction2(paragraph, question_set['problems'])  # 지문과 문제 리스트를 전달

        total_answers += predicted_answers
        # 예측 결과와 정답 비교
        for i, problem in enumerate(question_set['problems']):
            if predicted_answers[i] == problem['answer']:  # predicted_answers에서 해당 문제의 예측값 가져오기
                total_score += problem['score']
                correct_answers += 1
            total_questions += 1
            pbar.update(1)

# 결과 출력
accuracy = correct_answers / total_questions
print(f"Total Score: {total_score}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Answers: {correct_answers}/{total_questions}")



if total_score > 50:
    print("통과!", "total_score :", total_score)
else:
    print("미달성", "total_score :", total_score)


import pandas as pd

corrects = [ans['answer'] for dat in data for ans in dat['problems']]
res = pd.DataFrame({'모델의답변':total_answers, '실제답변':corrects[:len(total_answers)]})
display(res)


처리 중인 문제:   0%|          | 0/45 [00:00<?, ?it/s]

Raw GPT-4 response: 5
Raw GPT-4 response: 5
Raw GPT-4 response: 1
Raw GPT-4 response: 4
Raw GPT-4 response: 5
Rate limit reached. Waiting for 8.62 seconds.
Raw GPT-4 response: 3
Rate limit reached. Waiting for 8.57 seconds.
Raw GPT-4 response: 2
Rate limit reached. Waiting for 9.76 seconds.
Raw GPT-4 response: 5
Rate limit reached. Waiting for 7.87 seconds.
Raw GPT-4 response: 2
Rate limit reached. Waiting for 3.86 seconds.
Raw GPT-4 response: 4
Rate limit reached. Waiting for 5.74 seconds.
Raw GPT-4 response: 5
Rate limit reached. Waiting for 4.79 seconds.
Raw GPT-4 response: 3
Rate limit reached. Waiting for 5.04 seconds.
Raw GPT-4 response: 1
Rate limit reached. Waiting for 6.66 seconds.
Raw GPT-4 response: 3
Rate limit reached. Waiting for 8.05 seconds.
Raw GPT-4 response: 3
Rate limit reached. Waiting for 6.97 seconds.
Raw GPT-4 response: 4
Rate limit reached. Waiting for 7.38 seconds.
Raw GPT-4 response: 1
Rate limit reached. Waiting for 8.03 seconds.
Raw GPT-4 response: 4
Rate l

,모델의답변,실제답변
0,5,4
1,5,5
2,1,1
3,4,4
4,5,5
5,3,3
6,2,2
7,5,5
8,2,2
9,4,4


In [55]:

# 결과 출력
accuracy = correct_answers / total_questions
print(f"Total Score: {total_score}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Answers: {correct_answers}/{total_questions}")



if total_score > 50:
    print("통과!", "total_score :", total_score)
else:
    print("미달성", "total_score :", total_score)


import pandas as pd

corrects = [ans['answer'] for dat in data for ans in dat['problems']]
res = pd.DataFrame({'모델의답변':total_answers, '실제답변':corrects[:len(total_answers)]})
display(res)

Total Score: 79
Accuracy: 80.00%
Correct Answers: 36/45
통과! total_score : 79


,모델의답변,실제답변
0,5,4
1,5,5
2,1,1
3,4,4
4,5,5
5,3,3
6,2,2
7,5,5
8,2,2
9,4,4
